<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Semantic_Features_3DVision_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Our mapping pipeline is of the following structure:

1.   Detection of objects of certain object classes (e.g. traffic sign). Output: object bounding boxes
2.   Triangulation of objects. Output: object position relative to pose
3.   Creating map of objects. (And refining with filter, BA, etc.) Output: List of objects and their corresponding positions
4.   Visualizing map.





We start by loading the benchmark dataset. The Cityscapes dataset is used. 

Scripts for analyzing the dataset can be found here: https://github.com/mcordts/cityscapesScripts

How to download the zip files directly: https://towardsdatascience.com/download-city-scapes-dataset-with-script-3061f87b20d7



Download CityScape files: 

*   leftImg8bit_trainextra.zip
*   disparity_trainextra.zip
*   camera_trainextra.zip
*   vehicle_trainextra.zip

The files are unzipped into data/... respectively.









We start with the detection of objects of interest. Faster R-CNN is used for this task. (Julius)

In [0]:
# Get Results File from Julius (Run only once !)

import shutil
import os
import itertools
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

!pip install git+https://github.com/utiasSTARS/pykitti.git

import pykitti

!wget 'https://raw.githubusercontent.com/juliuserbach/Semantic-Features/master/results.json' -O results.json


path = "data_KITTI"
os.mkdir(path)
path = "data_KITTI/Numerical_Results"
os.mkdir(path)
shutil.copy("results.json", "data_KITTI/Numerical_Results")

# Download and unzip (run only once!).
PATH_TO_CALIB = "dataset/sequences/"
PATH_TO_POSE = "dataset/poses/"

!wget "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100433&authkey=APrMUGQyaB4np4Q" -O kitti_dataset.zip

!mkdir data_KITTI
!unzip -qq kitti_dataset.zip "$PATH_TO_CALIB"* -d data_KITTI
!unzip -qq kitti_dataset.zip "$PATH_TO_POSE"* -d data_KITTI

# And delete.
!rm kitti_dataset.zip


In [0]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
static_classes = []              

Loading an on COCO pre-trained Faster R-CNN 


Installing Seamless Scene Segmentation from https://github.com/mapillary/seamseg
For Seamless Scene Segmentation a pre-trained version on the Mapillary Dataset exists. Dataloaders for Cityscapes seem to exist, too.

It is a network for panoptic segmentation basen on Mask-R-CNN. Alternatively the panoptic variant in the detectron2 reopisitory can also be trained on Cityscapes or Mapillary. Dataloaders seem to exist for both Datasets. 

In [0]:
#@title
#!pip install git+https://github.com/mapillary/seamseg.git
#!pip install wget

#url = 'https://drive.google.com/file/d/1ULhd_CZ24L8FnI9lZ2H6Xuf03n6NA_-Y/view'
#!wget $url


These functions convert a difference in coordinates (lat., long.) to a difference in metric frame (x, y) and vice versa. Both conversions are dependent on the current latitude.
Under the assumption that we are in Europe, east of 0°!

Approximative conversions:

* Latitude: 1 deg = 110.574 km
* Longitude: 1 deg = 111.320*cos(latitude) km


In [0]:
# Mapping Algorithm Helper Functions

import math

# Convert degree to radian
def degree_to_rad_converter(degree_value):
  rad_value = degree_value * math.pi / 180
  return rad_value

# Calculate real 3D position
def pixel_to_3Dposition_converter(x, depth, focal):
  x_3D = x * depth / focal
  return x_3D

# Calculate box center to get average disparity
def boundaries_to_box_center(x_1, x_2):
  x_center = (x_1 + x_2)/2
  return x_center

# Get disparities of all points (pixels)
def disparity_catcher(x, y, img):
  # DATA_DIR="data/disparity/train_extra/schweinfurt"
  # im_directory = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
  # n_im = len(im_directory)


  disp = img[y, x]
  if disp != 0 and disp > 5125:
    disp = (disp - 1.) / 256.

    # print(disp)
    return disp
  else:
    return "nan"


# Convert disparity to actual depth information
def disparity_to_depth(disparity_box_center):
  if disparity_box_center > 0:
    depth_center = baseline * focal / disparity_box_center
  else:
    depth_center = 100
  return depth_center

# Transformation to world frame coordinates
# _3dImage	=	cv.reprojectImageTo3D('data/disparity/train_extra/schweinfurt/schweinfurt_000000_000000_disparity.png', Q)


In [0]:
# Change this to the directory where you store KITTI data

basedir = '/content/data_KITTI/dataset'

# Specify the dataset to load

sequence = '04'

# Load the data. Optionally, specify the frame range to load.

dataset = pykitti.odometry(basedir, sequence)

i = 0
origin_pose =[]
origin_rot = []

# Save all absolute positions and Rot Matrices

for i in range(len(dataset.poses)):
  origin_pose.append(dataset.poses[i][0:3,3])
  origin_rot.append(dataset.poses[i][0:3,0:3])

print(origin_pose[0])
print(origin_rot[0])

# To do:
# - Get Position of car
# - Get Camera Calibration data of left camera
# - Do normal camera calibration but with new data
# - Get disparity using new file
# - Add new position data onto coordinates



In [0]:
# Get Camera Calibration Data

P_cam2 = dataset.calib.P_rect_20
baseline = dataset.calib.b_rgb
fx = P_cam2[0, 0]
fy = P_cam2[1, 1]
u_0 = P_cam2[0, 2]
v_0 = P_cam2[1, 2]
focal = fx


In [0]:
# Mapping Algorithm -> Associates objects with respective coordinates in world coordinate frame and stores them in vector containing Long, Lat,
#                      Height, class and corresponding image ID

import pandas as pd
import math
import csv
import pandas as pd
import cv2
import glob

# Ask which disparity version to take from disparity matrix. Options are: 0 for median or 1 for maximum

option = input("Enter your option (0 = median, 1 = maximum): ")
option = int(option)

k = 0 # Images
t = 0 # Total valid object number

x_3D_rel_car = []
y_3D_rel_car = []
z_3D_rel_car = []
x = []
y = []
relz = []
semanticlabel = []
imageid = []

results_directory = '/content/data_KITTI/Numerical_Results/results.json'

# Load json from path
with open(results_directory) as json_file:
  data = json.load(json_file)
  for i in range(len(data['results'])):
    print("Processing image number", k)
    # print(x_3D_rel_car)
    n_objects = int(len(data['results'][k]['classes']))
    # Get image number
    image_no = data['results'][k]['image_id'][3:6]
    print(image_no)
    im_no = int(image_no)
    # disparity_data_path = 'data/disparity/train_extra/schweinfurt/schweinfurt_000000_000'+ image_no + '_disparity.png'
    disparity_data_path = 'data_KITTI/dataset/sequences/04/disparity_2/' + image_no + 'raw.png'

    # Load image from path
    img = cv2.imread(disparity_data_path, cv2.IMREAD_UNCHANGED)

    for o in range(n_objects):
      # Get coordinates in image frame
      v1 = data['results'][k]['boxes'][o][0]
      u1 = data['results'][k]['boxes'][o][1]
      v2 = data['results'][k]['boxes'][o][2]
      u2 = data['results'][k]['boxes'][o][3]

      # Calculate center point in image frame
      u_cent = int(boundaries_to_box_center(u1, u2))
      v_cent = int(boundaries_to_box_center(v1, v2))

      # Get and write all disparities 1/10th of box-length around center point in matrix
      disp_matrix = np.zeros(shape=(int(abs(u2 - u1)),int(abs(v2 - v1))))

      for y_coord in range(int(abs((v2 - v1)))):
        for x_coord in range(int(abs((u2 - u1)))):
          try:
            disp_matrix[x_coord][y_coord] = disparity_catcher(u1 + x_coord, v1 + y_coord, img)
          except TypeError:
            disp_matrix[x_coord][y_coord] = 0
        
      # Take median of disparity values
      
      if option == 0:
        med_disp = np.nanmedian(disp_matrix, axis=None, out=None, overwrite_input=False, keepdims=False)
      if option == 1:
        try:
          med_disp = np.nanmax(disp_matrix, axis=None, out=None, keepdims=False)
        except RuntimeWarning:
          med_disp = np.nanmax(disp_matrix, axis=None, out=None, keepdims=False)
        #else:
          #med_disp = np.nanmax(disp_matrix, axis=None, out=None, keepdims=False)

      if med_disp != 0:

        # Get disparity of center point in image frame
        # disp_cent = disparity_catcher(u_cent, v_cent, k)

        # Calculate depth from disparity
        depth_med = disparity_to_depth(med_disp)

        # Transform coordinates from image plane to camera frame
        x1 = depth_med * (u1 - u_0) / f_x
        y1 = depth_med * (v1 - v_0) / f_y
        x2 = depth_med * (u2 - u_0) / f_x
        y2 = depth_med * (v2 - v_0) / f_y

        # Calculate box center in camera frame
        x_cent = boundaries_to_box_center(x1, x2)
        y_cent = boundaries_to_box_center(y1, y2)

        # Save object coordinates in camera frame
        x_3D_rel_car.append(x_cent)
        y_3D_rel_car.append(y_cent)
        z_3D_rel_car.append(depth_med)

        # Transform object coordinates to world frame
        input_vector = np.array([[x_3D_rel_car[t]], [y_3D_rel_car[t]], [z_3D_rel_car[t]]])
        try:
          output_vector = origin_rot[im_no] * input_vector + origin_pose[im_no]
        except RuntimeWarning:
          output_vector = [0, 0, 0]

        
        # dataframe
        x.append(output_vector[0])
        y.append(output_vector[1])
        relz.append(output_vector[2])
        semanticlabel.append(data['results'][k]['classes'][o])
        imageid.append(data['results'][k]['image_id'][3:6])
      
        t += 1
    k += 1

  mapping_output_df = pd.DataFrame({'imageID': imageid, 'x': x, 'y': y, 'z': relz, 'semanticlabel': semanticlabel})

  for i in mapping_output_df:
    mapping_output_df[i] = np.squeeze(mapping_output_df[i].tolist())


In [0]:
### Writes Data to DataFrame and to csv file

# Create DataFrame 


# Print the output
print(mapping_output_df)

# Write output to csv file
mapping_output_df.to_csv(r'data_KITTI/mapping_output.csv', index = False)